# amazon-sagemaker-inference-using-tenstorhub-ready-model

This notebook is for building and deploying the code-sample as a BYOC (Bring your own container) as an Amazon SageMaker endpoint for object detection.

The model is a ready model downloaded directly from [Tensor Hub](https://www.tensorflow.org/hub/)

To use this notebook, you need to run it in AWS, preferably in Amazon SageMaker, with permissions to Amazon Elastic Container Registry, and Amazon Sagemaker

## Prerequisite

Lets configure some variables we will use later

`SVC` will be the name of the ECR container repository name
`TAG` will be the tag of the container image for proper versioning

In [ ]:
# ECR Repository name
SVC='amazon-sagemaker-inference-using-tenstorhub-ready-model'
TAG='1.0'

We will add additional variables that will be used next

In [ ]:
!pip install sagemaker

import boto3
import sagemaker

sess = sagemaker.Session()
s3_bucket = sess.default_bucket()
sts = boto3.client("sts")
account_id = sts.get_caller_identity()["Account"]
aws_region = boto3.session.Session().region_name
ecr_url = "{}.dkr.ecr.{}.amazonaws.com".format(account_id, aws_region)

Create a docker repository in [AWS ECR](https://aws.amazon.com/ecr/), the repository will store the docker image that we are about to build and push to the repository.

In [ ]:
!aws ecr --region $aws_region create-repository --repository-name $SVC > /dev/null

## Build

We will now build the a docker image according to [`Dockerfile`](./Dockerfile) and we will tag the container image in the build command

In [ ]:
full_repository_url = "{}/{}:{}".format(ecr_url, SVC, TAG)
!docker build -t $full_repository_url .

In order to push to Amazon ECR, we will need to authenticate using a temporary login password from IAM, this password will allow us to securely push the docker image to Amazon ECR

In [ ]:
!aws ecr get-login-password --region $aws_region | docker login --username AWS --password-stdin $ecr_url

Push docker image to the ECR repository

In [ ]:
!docker push $full_repository_url

## Deploy

After we finished building the container image, we can start working on Amazon SageMaker.
We will need to follow couple of steps:

1. Creating a model that points to the container image
2. Creating a model endpoint configuration that contains the model, instance type etc...
3. Deploy - creating an endpoint from the model and endpoint configuration


First we will use the `create_model` to create the docker image that we stored in ECR and a model in Amazon SageMaker.
We will print out the result to see what Amazon SageMaker API returns.

For more details on `create_model` go to the following [documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model)

In [ ]:
from sagemaker import get_execution_role

role = get_execution_role()

client = boto3.client('sagemaker')

response = client.create_model(
    ModelName=SVC,
    PrimaryContainer={
        'ContainerHostname': SVC,
        'Image': full_repository_url,
        'ImageConfig': {
            'RepositoryAccessMode': 'Platform',
            'RepositoryAuthConfig': {
                'RepositoryCredentialsProviderArn': role
            }
        }
    },
    ExecutionRoleArn=role,
    EnableNetworkIsolation=False
)
print(response)

After we've created the model using `create_model`, now we will create a model config using `create_endpoint_config`.
A model config will have the `ModelName` that we've created before, with additional settings like `InstanceType` the amount of instances to Launch.

We will print out the conf response to see what the API returns.

For more details on `create_endpoint_config` go to the following [documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config)

In [ ]:
conf = client.create_endpoint_config(
    EndpointConfigName=SVC,
    ProductionVariants=[{
        'VariantName': 'default',
        'ModelName': SVC,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'InstanceType': 'ml.g4dn.xlarge'
    }]
)
print(conf)

After we have created a model and an endpoint config, we can launch a new Amazon SageMaker that will use the container image we've built, with the endpoint config we have set.

We will print the endpoint response to see what the API returns

For more details on `create_endpoint` go to the following [documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint)

In [ ]:
endpoint = client.create_endpoint(
    EndpointName=SVC,
    EndpointConfigName=SVC
)
print(endpoint)

The deployment of a new endpoint takes couple of minutes, lets execute the following code block to wait until the endpoint is ready.

A ready SageMaker endpoint responds `200 OK` on `/ping` requests from SageMaker, once the endpoint is ready, it's status will be `InService` than we can start invoking the endpoint for inference 

In [ ]:
# Wait until the endpoint is InService
import time

endpoint_info = client.describe_endpoint(EndpointName=SVC)
endpoint_status = endpoint_info["EndpointStatus"]

while endpoint_status == "Creating":
    endpoint_info = client.describe_endpoint(EndpointName=SVC)
    endpoint_status = endpoint_info["EndpointStatus"]
    print("Endpoint status:", endpoint_status)
    if endpoint_status == "Creating":
        time.sleep(10)

print("Endpoint is in {}".format(endpoint_status))

## Testing the endpoint

The endpoint was built to download an images from Amazon S3 as an input.

So let's upload the `./Naxos_Taverna.jpg` image to the default Amazon SageMaker bucket that we configured when starting this notebook as `s3_bucket` and then we will invoke the endpoint with the appropriate information.

In [ ]:
!aws s3 cp ./Naxos_Taverna.jpg s3://$s3_bucket/test/

Now we can invoke the endpoint, the endpoint will do as follow:

1. Download the image to its temp storage
2. Run inference on the image
3. Delete the image
4. Return a `json` with all the objects detected, their confidence store, and their detection boxes bounding box coordinates.

We will print the model prediction response.

>Note, that the first invocation is slow, its the first time that `tensorflow_hub` loads the model. You can invoke it twice, to see how it will perform.

In [ ]:
import json

runtime = boto3.client('sagemaker-runtime')

payload = {
    "s3_bucket": s3_bucket,
    "key_prefix": 'test',
    "file_name": 'Naxos_Taverna.jpg'
}

response = runtime.invoke_endpoint(
    EndpointName=SVC,
    Body=json.dumps(payload),
    ContentType="application/json"
)

model_prediction = json.loads(response["Body"].read())
print(model_prediction)

Let's see the image we are going to use for this sample

In [ ]:
from IPython.core.display import HTML

HTML('<img src="Naxos_Taverna.jpg" alt="Naxos_Taverna.jpg"'
     '<figcaption>Naxos_Taverna.jpg</figcaption>The image has been downloaded from '
     'https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg, '
     '<a href="https://en.wikipedia.org/wiki/GNU_Free_Documentation_License">License</a>')

Now that we have the results, let's test how the model performed detecting objects in the image.

We will use `pillow` to draw on the image, and we will only draw a bounding box for objects with a confidence score higher than 0.5

In [ ]:
!pip install pillow

from PIL import Image, ImageDraw, ImageColor, ImageFont
import numpy as np

conf_score = 0.5

def draw_bounding_box_on_image(image, ymin, xmin, ymax, xmax, color, font, thickness=1, display_str_list=()):
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
              (left, top)],
              width=thickness,
              fill=color)

    # If the total height of the display strings added to the top of the bounding
    # box exceeds the top of the image, stack the strings below the bounding box
    # instead of above.
    display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
    # Each display_str has a top and bottom margin of 0.05x.
    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = top + total_display_str_height
    # Reverse list and print from bottom to top.
    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                        (left + text_width, text_bottom)],
                       fill=color)
        draw.text((left + margin, text_bottom - text_height - margin),
                  display_str,
                  fill='red',
                  font=font)
        text_bottom -= text_height - 2 * margin


def draw_bounding_box(image_path, inferred_image_result):
    colors = list(ImageColor.colormap.values())
    font = ImageFont.load_default()
    
    with Image.open(image_path) as img:
        for i in range(0, len(inferred_image_result)):
            if float(inferred_image_result[i]['confidence']) >= conf_score:
                ymin = float(inferred_image_result[i]['ymin'])
                xmin = float(inferred_image_result[i]['xmin'])
                ymax = float(inferred_image_result[i]['ymax'])
                xmax = float(inferred_image_result[i]['xmax'])
                obj_class = inferred_image_result[i]["class"]
                obj_confidence = float(inferred_image_result[i]["confidence"])

                # set bounding box display string
                display_str = "{}: {}%".format(obj_class, obj_confidence * 100)
                
                draw_bounding_box_on_image(img, ymin, xmin, ymax, xmax, colors[i], font, display_str_list=[display_str])

    img.save('./Naxos_Taverna-boxed.jpg')

Now we will call the functions we configured with the image [`/Naxos_Taverna.jpg`](./Naxos_Taverna.jpg) and the `model_prediction` results

In [ ]:
draw_bounding_box('./Naxos_Taverna.jpg', model_prediction)

Lets see the image with the bounding boxes

In [ ]:
from IPython.core.display import HTML

HTML('<img src="Naxos_Taverna-boxed.jpg" alt="Naxos_Taverna-boxed.jpg"'
     '<figcaption>Naxos_Taverna-boxed.jpg</figcaption>The image has been downloaded from '
     'https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg, '
     '<a href="https://en.wikipedia.org/wiki/GNU_Free_Documentation_License">License</a>')

### Clean up

Make sure to run clean up to avoid unplanned costs of the running endpoint.

In [ ]:
# Amazon SageMaker

client.delete_endpoint(EndpointName=SVC)
client.delete_endpoint_config(EndpointConfigName=SVC)
client.delete_model(ModelName=SVC)


In [ ]:
# ECR
!aws ecr delete-repository --region $aws_region --repository-name $SVC --force